## Requirement 1: Stochatic Environment
**Pricing algorithm: Build a pricing strategy using the continuous set of prices p ∈ [0, 1] and Gaussian Processes.**\
Bidding algorithm: Consider a sequence of second-price auctions. Build two learning algorithms to deal with the bidding problem:
- primal-dual algorithm for truthful auctions
- UCB-like algorithm

### Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm 

### Stochastic Pricing Environment

In [ ]:
class StochasticPricingEnvironment:
    def __init__(self, cost, max_price=1.0, model='linear', alpha=None, beta=None):
        self.cost = cost
        self.max_price = max_price
        self.model = model
        self.alpha = alpha
        self.beta = beta
        # self.conversion_probability = StochasticPricingEnvironment.generate_probability(model, alpha, beta)
        np.random.seed(42)

    # given settled price and number customer
    # returns a sample of number of sale and profit
    def round(self, price_t, n_costumer_t):
        # number of sale at time t
        number_sale_t = np.random.binomial(n_costumer_t, StochasticPricingEnvironment.generate_probability(self.model, self.alpha, self.beta, price_t))
        
        # profit at time t
        profit_t = (price_t - self.cost)*number_sale_t

        return number_sale_t, profit_t

    # return a probability function that 
    # takes as input the selling price
    # return as output the probability of selling 
    @staticmethod
    def generate_probability(model, a, b, price):
        def isNotNone(s,d):
            if s is None:
                return d
            else:
                return s
        
        if model == 'linear':
            alpha = isNotNone(a, 1) # Intercept
            beta = isNotNone(b, -1)  # Slope
            purchase_probabilities = alpha + beta * price
            
        elif model == 'logit':
            alpha = isNotNone(a, 0)
            beta = isNotNone(b, -5)
            purchase_probabilities = 1 / (1 + np.exp(-(alpha + beta * price)))
            
        elif model == 'probit':
            alpha = isNotNone(a, 0)
            beta = isNotNone(b, -5)
            purchase_probabilities = norm.cdf(alpha + beta * price)
            
        elif model == 'KERNEL':
            purchase_probabilities = np.abs(np.sin(2 * np.pi * price) * np.exp(-price * 5) + price * .10) + .1

        return purchase_probabilities


### Pricing Agent

#### Radial Basis Function Kernel

In [ ]:
class RBFGaussianProcess:
    def __init__(self, scale=1, reg=1e-2):
        self.scale = scale 
        self.reg = reg
        self.k_xx_inv = None

    def rbf_kernel_incr_inv(self, B, C, D):
        temp = np.linalg.inv(D - C @ self.k_xx_inv @ B)
        block1 = self.k_xx_inv + self.k_xx_inv @ B @ temp @ C @ self.k_xx_inv
        block2 = - self.k_xx_inv @ B @ temp
        block3 = - temp @ C @ self.k_xx_inv
        block4 = temp
        res1 = np.concatenate((block1, block2), axis=1)
        res2 = np.concatenate((block3, block4), axis=1)
        res = np.concatenate((res1, res2), axis=0)
        return res

    def rbf_kernel(self, a, b):
        a_ = a.reshape(-1, 1)
        b_ = b.reshape(-1, 1)
        output = -1 * np.ones((a_.shape[0], b_.shape[0]))
        for i in range(a_.shape[0]):
            output[i, :] = np.power(a_[i] - b_, 2).ravel()
        return np.exp(-self.scale * output)
    
    def fit(self, x=np.array([]), y=np.array([])):
        x,y = np.array(x),np.array(y)
        if self.k_xx_inv is None:
            self.y = y.reshape(-1,1)
            self.x = x.reshape(-1,1)
            k_xx = self.rbf_kernel(self.x, self.x) + self.reg * np.eye(self.x.shape[0])
            self.k_xx_inv = np.linalg.inv(k_xx)
        else:
            B = self.rbf_kernel(self.x, x)
            self.x = np.vstack((self.x, x))
            self.y = np.vstack((self.y, y))
            self.k_xx_inv = self.rbf_kernel_incr_inv(B, B.T, np.array([1 + self.reg]))

        return self

    def predict(self, x_predict):
        k = self.rbf_kernel(x_predict, self.x)

        mu_hat = k @ self.k_xx_inv @ self.y
        sigma_hat = 1 - np.diag(k @ self.k_xx_inv @ k.T)

        return mu_hat.ravel(), sigma_hat.ravel()

#### Gaussian Process Agent with RBF kernel and UCB algorithm

In [ ]:
# From the agent's point of view, action set is [0,1]. If the actual actions are outside this
# set, we can always perform a rescaling outside the class.
class GPUCBAgent:
    def __init__(self, T, discretization=100):
        self.T = T
        self.arms = np.linspace(0, 1, discretization)
        self.gp = RBFGaussianProcess(scale=2).fit()
        self.a_t = None
        self.action_hist = np.array([])
        self.reward_hist = np.array([])
        self.mu_t = np.zeros(discretization)
        self.sigma_t = np.zeros(discretization)
        self.gamma = lambda t: np.log(t+1)**2 
        self.beta = lambda t: 1 + 0.5*np.sqrt(2 * (self.gamma(t) + 1 + np.log(T)))
        self.N_pulls = np.zeros(discretization)
        self.t = 0
    
    def pull_arm(self):
        self.mu_t, self.sigma_t = self.gp.predict(self.arms) 
        #    averages of GP  +  estimation uncertainty
        ucbs = self.mu_t + self.beta(self.t) * self.sigma_t
        self.a_t = np.argmax(ucbs)
        return self.arms[self.a_t]
    
    def update(self, profit_per_user_t):
        self.N_pulls[self.a_t] += 1
        self.action_hist = np.append(self.action_hist, self.arms[self.a_t])
        self.reward_hist = np.append(self.reward_hist, profit_per_user_t)
        self.gp = self.gp.fit(self.arms[self.a_t], profit_per_user_t)
        self.t += 1

### Second Price Auctions

In [ ]:
class SecondPriceAuction:
    def __init__(self, ctrs):
        # ctr = click though rate = lambda * q
        self.ctrs = ctrs
        self.n_adv = len(self.ctrs)
    
    def get_winners(self, bids):
        # sort not by bids, but estimated values
        adv_values = self.ctrs*bids
        adv_ranking = np.argsort(adv_values)
        winner = adv_ranking[-1]
        return winner, adv_values
    
    def get_payments_per_click(self, winners, values, bids):
        adv_ranking = np.argsort(values)
        second = adv_ranking[-2]
        payment = values[second]/self.ctrs[winners]
        return payment.round(2)
    
    def round(self, bids):
        # given bids, return winner(s) and the estimated values (product of 3 terms : lambda, q, bid) of winner(s)
        winners, values = self.get_winners(bids) # allocation mechanism!
        payments_per_click = self.get_payments_per_click(winners, values, bids)
        return winners, payments_per_click

### Bidding Agents

#### Primal-dual algorithm -> Multiplicative Pacing

In [ ]:
class MultiplicativePacingAgent:
    def __init__(self, valuation, budget, T, eta):
        self.valuation = valuation
        self.budget = budget
        self.eta = eta # learning rate
        self.T = T
        self.rho = self.budget/self.T
        self.lmbd = 1  # shouldn't this be initialized with 0?
        self.t = 0

    def bid(self):
        if self.budget < 1:
            return 0
        return self.valuation/(self.lmbd+1)
    
    def update(self, f_t, c_t):
        self.lmbd = np.clip(self.lmbd-self.eta*(self.rho-c_t), 
                            a_min=0, a_max=1/self.rho)
        self.budget -= c_t

#### UCB-like algorithm -> UCB-Multiplicative Pacing

In [ ]:
class UCBMultiPaceAgent:
    def __init__(self, valuation, budget, T, eta, number_rhos, range=1):
        self.valuation = valuation
        self.budget = budget
        self.eta = eta # learning rate
        self.T = T
        self.range = range
        self.used_rho = None
        self.number_rhos = number_rhos
        self.average_rewards = np.zeros(number_rhos)
        self.N_pulls = np.zeros(number_rhos)
        self.rhos = np.linspace(budget/T, valuation, number_rhos)
        self.lmbds = np.ones(number_rhos)
        self.t = 0

    def bid(self):
        # play every arm once
        if self.t < self.number_rhos:
            self.used_rho = self.t 
        else:
            # compute UCB for every arms, choose arm with highest UCB
            ucbs = self.average_rewards + self.range*np.sqrt(2*np.log(self.T)/self.N_pulls)
            self.used_rho = np.argmax(ucbs)
    
        if self.budget < 1:
            return 0
        return self.valuation/(self.lmbds[self.used_rho]+1)
    
    def update(self, f_t, c_t):
        self.N_pulls[self.used_rho] += 1
        # update only used rho's average reward
        self.average_rewards[self.used_rho] += (f_t - self.average_rewards[self.used_rho])/self.N_pulls[self.used_rho]
        # update all rhos's lmbd according to c_t and their rho value
        self.lmbds = np.clip(self.lmbds-self.eta*(self.rhos-c_t), 
                            a_min=0, a_max=1/self.rhos)
        self.budget -= c_t
        self.t += 1

#### UCB-like algorithm -> simple UCB

In [ ]:
class UCBAgent:
    def __init__(self, valuation, budget, T, K=100, range=1):
        self.T = T
        self.valuation = valuation
        self.budget = budget
        self.K = K
        self.range = range
        self.arms = np.linspace(0, valuation, K)
        self.average_rewards = np.zeros(K)
        self.N_pulls = np.zeros(K)
        self.a_t = None
        self.t = 0

    def bid(self):
        # if budget depleted
        if self.budget < 1:
            return 0
        
        # play every arm at least once
        if self.t < self.K:
            self.a_t = self.t
        else:
            # compute UCB for every arms, choose arm with highest UCB
            ucbs = self.average_rewards + self.range*np.sqrt(2*np.log(self.T)/self.N_pulls)
            self.a_t = np.argmax(ucbs)
        return self.arms[self.a_t]

    def update(self, f_t, c_t):
        self.t += 1
        self.N_pulls[self.a_t] += 1
        self.average_rewards[self.a_t] += (f_t - self.average_rewards[self.a_t])/self.N_pulls[self.a_t]
        self.budget -= c_t